 # Analyzing NYC High School Data
 ---
 This is a guided project part of the Dataquest.io courses. The goal of this project is to examine NYC high school data to find out what causes high SAT scores.

 There is a good amount of public data available regarding education statistics and SAT scores. The only issue is that these are available in separate sources. Therefore the first part of this project we will need to read in the data and clean it up enough so that all of the datasets can be combined.

 Additionally, we will also experiment with using the Altair library for visualizations.

 # Read in the data

In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib
import altair as alt
import geopandas as gpd
import gpdvega

data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

data = {}

for f in data_files:
    d = pd.read_csv("data/schools/{0}".format(f))
    data[f.replace(".csv", "")] = d


In [4]:
all_survey = pd.read_csv("data/schools/survey_all.txt", delimiter="\t", encoding='windows-1252')
d75_survey = pd.read_csv("data/schools/survey_d75.txt", delimiter="\t", encoding='windows-1252')
survey = pd.concat([all_survey, d75_survey], axis=0, sort=False)

survey["DBN"] = survey["dbn"]

survey_fields = [
    "DBN", 
    "rr_s", 
    "rr_t", 
    "rr_p", 
    "N_s", 
    "N_t", 
    "N_p", 
    "saf_p_11", 
    "com_p_11", 
    "eng_p_11", 
    "aca_p_11", 
    "saf_t_11", 
    "com_t_11", 
    "eng_t_11", 
    "aca_t_11", 
    "saf_s_11", 
    "com_s_11", 
    "eng_s_11", 
    "aca_s_11", 
    "saf_tot_11", 
    "com_tot_11", 
    "eng_tot_11", 
    "aca_tot_11",
]
survey = survey.loc[:,survey_fields]
data["survey"] = survey


In [5]:
survey.columns


Index(['DBN', 'rr_s', 'rr_t', 'rr_p', 'N_s', 'N_t', 'N_p', 'saf_p_11',
       'com_p_11', 'eng_p_11', 'aca_p_11', 'saf_t_11', 'com_t_11', 'eng_t_11',
       'aca_t_11', 'saf_s_11', 'com_s_11', 'eng_s_11', 'aca_s_11',
       'saf_tot_11', 'com_tot_11', 'eng_tot_11', 'aca_tot_11'],
      dtype='object')

In [6]:
all_survey = pd.read_csv("data/schools/survey_all.txt", delimiter="\t", encoding='windows-1252')
d75_survey = pd.read_csv("data/schools/survey_d75.txt", delimiter="\t", encoding='windows-1252')
survey = pd.concat([all_survey, d75_survey], axis=0, sort=False)

survey["DBN"] = survey["dbn"]
sruvey_fields = survey.columns

survey = survey.loc[:,survey_fields]
data["survey"] = survey


 Our datasets are not encoded in a standard format. There isn't really a way to tell except to try loading and see how the data looks like.

In [7]:
data["survey"].columns


Index(['DBN', 'rr_s', 'rr_t', 'rr_p', 'N_s', 'N_t', 'N_p', 'saf_p_11',
       'com_p_11', 'eng_p_11', 'aca_p_11', 'saf_t_11', 'com_t_11', 'eng_t_11',
       'aca_t_11', 'saf_s_11', 'com_s_11', 'eng_s_11', 'aca_s_11',
       'saf_tot_11', 'com_tot_11', 'eng_tot_11', 'aca_tot_11'],
      dtype='object')

In [8]:
all_survey = pd.read_csv("data/schools/survey_all.txt", delimiter="\t", encoding='windows-1252')
d75_survey = pd.read_csv("data/schools/survey_d75.txt", delimiter="\t", encoding='windows-1252')
survey = pd.concat([all_survey, d75_survey], axis=0, sort=False)

survey["DBN"] = survey["dbn"]
sruvey_fields = survey.columns

survey = survey.loc[:,survey_fields]
data["survey"] = survey


 # Analyzing NYC High School Data
 ---
 This is a guided project part of the Dataquest.io courses. The goal of this project is to examine NYC high school data to find out what causes high SAT scores.

 There is a good amount of public data available regarding education statistics and SAT scores. The only issue is that these are available in separate sources. Therefore the first part of this project we will need to read in the data and clean it up enough so that all of the datasets can be combined.

 Additionally, we will also experiment with using the Altair library for visualizations.

 ## Load and Transform Data

In [25]:
import pandas as pd
import numpy as np
import re
import matplotlib
import altair as alt
import geopandas as gpd
import gpdvega

data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

data = {}

for f in data_files:
    d = pd.read_csv("data/schools/{0}".format(f))
    data[f.replace(".csv", "")] = d


 Our datasets are not encoded in a standard format. There isn't really a way to tell except to try loading and see how the data looks like.

In [26]:
all_survey = pd.read_csv("data/schools/survey_all.txt", delimiter="\t", encoding='windows-1252')
d75_survey = pd.read_csv("data/schools/survey_d75.txt", delimiter="\t", encoding='windows-1252')
survey = pd.concat([all_survey, d75_survey], axis=0, sort=False)

survey["DBN"] = survey["dbn"]
sruvey_fields = survey.columns

survey = survey.loc[:,survey_fields]
data["survey"] = survey

 When working with multiple datasets, a common field is required to join them together into one set. In many datasets, it may not be obvious what that key field is.
 Sometimes, a different key is required for each pairing of sets. We can use DBN as our key column for all sets. This will require additional cleaning though.

In [27]:
data["hs_directory"]["DBN"] = data["hs_directory"]["dbn"]

def pad_csd(num):
    string_representation = str(num)
    if len(string_representation) > 1:
        return string_representation
    else:
        return "0" + string_representation
    
data["class_size"]["padded_csd"] = data["class_size"]["CSD"].apply(pad_csd)
data["class_size"]["DBN"] = data["class_size"]["padded_csd"] + data["class_size"]["SCHOOL CODE"]


In [28]:
cols = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score', 'SAT Writing Avg. Score']
for c in cols:
    data["sat_results"][c] = pd.to_numeric(data["sat_results"][c], errors="coerce")

data['sat_results']['sat_score'] = data['sat_results'][cols[0]] + data['sat_results'][cols[1]] + data['sat_results'][cols[2]]

def find_lat(loc):
    coords = re.findall("\(.+, .+\)", loc)
    lat = coords[0].split(",")[0].replace("(", "")
    return lat

def find_lon(loc):
    coords = re.findall("\(.+, .+\)", loc)
    lon = coords[0].split(",")[1].replace(")", "").strip()
    return lon

data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(find_lat)
data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(find_lon)

data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"], errors="coerce")
data["hs_directory"]["lon"] = pd.to_numeric(data["hs_directory"]["lon"], errors="coerce")


 # Analyzing NYC High School Data
 ---
 This is a guided project part of the Dataquest.io courses. The goal of this project is to examine NYC high school data to find out what causes high SAT scores.

 There is a good amount of public data available regarding education statistics and SAT scores. The only issue is that these are available in separate sources. Therefore the first part of this project we will need to read in the data and clean it up enough so that all of the datasets can be combined.

 Additionally, we will also experiment with using the Altair library for visualizations.

 ## Load and Transform Data

In [13]:
import pandas as pd
import numpy as np
import re
import matplotlib
import altair as alt
import geopandas as gpd
import gpdvega

data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

data = {}

for f in data_files:
    d = pd.read_csv("data/schools/{0}".format(f))
    data[f.replace(".csv", "")] = d


 Our datasets are not encoded in a standard format. There isn't really a way to tell except to try loading and see how the data looks like.

In [14]:
all_survey = pd.read_csv("data/schools/survey_all.txt", delimiter="\t", encoding='windows-1252')
d75_survey = pd.read_csv("data/schools/survey_d75.txt", delimiter="\t", encoding='windows-1252')
survey = pd.concat([all_survey, d75_survey], axis=0, sort=False)

survey["DBN"] = survey["dbn"]
sruvey_fields = survey.columns

survey = survey.loc[:,survey_fields]
data["survey"] = survey

 When working with multiple datasets, a common field is required to join them together into one set. In many datasets, it may not be obvious what that key field is.
 Sometimes, a different key is required for each pairing of sets. We can use DBN as our key column for all sets. This will require additional cleaning though.

In [15]:
data["hs_directory"]["DBN"] = data["hs_directory"]["dbn"]

def pad_csd(num):
    string_representation = str(num)
    if len(string_representation) > 1:
        return string_representation
    else:
        return "0" + string_representation
    
data["class_size"]["padded_csd"] = data["class_size"]["CSD"].apply(pad_csd)
data["class_size"]["DBN"] = data["class_size"]["padded_csd"] + data["class_size"]["SCHOOL CODE"]


 Next is to convert our numeric columns to actually be numeric.

In [16]:
cols = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score', 'SAT Writing Avg. Score']
for c in cols:
    data["sat_results"][c] = pd.to_numeric(data["sat_results"][c], errors="coerce")

data['sat_results']['sat_score'] = data['sat_results'][cols[0]] + data['sat_results'][cols[1]] + data['sat_results'][cols[2]]

def find_lat(loc):
    coords = re.findall("\(.+, .+\)", loc)
    lat = coords[0].split(",")[0].replace("(", "")
    return lat

def find_lon(loc):
    coords = re.findall("\(.+, .+\)", loc)
    lon = coords[0].split(",")[1].replace(")", "").strip()
    return lon

data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(find_lat)
data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(find_lon)

data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"], errors="coerce")
data["hs_directory"]["lon"] = pd.to_numeric(data["hs_directory"]["lon"], errors="coerce")


 Next, we'll do some some filtering and grouping to get the data in a more analysis-friendly format.

In [17]:
class_size = data["class_size"]
class_size = class_size[class_size["GRADE "] == "09-12"]
class_size = class_size[class_size["PROGRAM TYPE"] == "GEN ED"]

class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)
data["class_size"] = class_size

data["demographics"] = data["demographics"][data["demographics"]["schoolyear"] == 20112012]

data["graduation"] = data["graduation"][data["graduation"]["Cohort"] == "2006"]
data["graduation"] = data["graduation"][data["graduation"]["Demographic"] == "Total Cohort"]


In [18]:
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']

for col in cols:
    data["ap_2010"][col] = pd.to_numeric(data["ap_2010"][col], errors="coerce")


 Finally, we'll use `merge` to join the datasets together. Note that the `how` argument means that all rows from the `combined` dataframe will be retained even if a match in the `ap_2010` is not found.

In [19]:
combined = data["sat_results"]

combined = combined.merge(data["ap_2010"], on="DBN", how="left")
combined = combined.merge(data["graduation"], on="DBN", how="left")

to_merge = ["class_size", "demographics", "survey", "hs_directory"]

for m in to_merge:
    combined = combined.merge(data[m], on="DBN", how="inner")

combined = combined.fillna(combined.mean())
combined = combined.fillna(0)


 # Add a school district column for mapping

 We add a school district column as that will be helpful for when we map the data.

In [20]:
def get_first_two_chars(dbn):
    return dbn[0:2]

combined["school_dist"] = combined["DBN"].apply(get_first_two_chars)

 ## Find correlations

In [21]:
correlations = combined.corr()
correlations = correlations["sat_score"].abs()
print(correlations.sort_values(ascending=False).head(20))


sat_score                               1.000000
SAT Writing Avg. Score                  0.987771
SAT Critical Reading Avg. Score         0.986820
SAT Math Avg. Score                     0.972643
frl_percent                             0.722225
white_per                               0.620718
asian_per                               0.570730
AP Test Takers                          0.523140
Total Exams Taken                       0.514333
asian_num                               0.475445
Number of Exams with scores 3 4 or 5    0.463245
white_num                               0.449559
sped_percent                            0.448170
N_s                                     0.423463
N_p                                     0.421530
total_students                          0.407827
ell_percent                             0.398750
hispanic_per                            0.396985
NUMBER OF STUDENTS / SEATS FILLED       0.394626
female_num                              0.388631
Name: sat_score, dty

 Taking the absolute value of coefficient, we can get a quick snapshot of how each value correlates with SAT scores. Keep in mind that some of these may be negatively correlated since we took the absolute value. Some quick notes:
 - Scores on the individual portions correlate highly, but is not useful to us since the individual portions make up the total SAT score
 - A high number of students will naturally lead to higher scores due to more chances for a high scores to occur

 ## Plotting survey correlations

In [22]:
# Remove DBN since it's a unique identifier, not a useful numerical value for correlation.
survey_fields.remove("DBN")

In [23]:
get_ipython().run_line_magic('matplotlib', 'inline')
combined.corr()["sat_score"][survey_fields] #.plot.bar()

rr_s          0.232199
rr_t         -0.023386
rr_p          0.047925
N_s           0.423463
N_t           0.291463
N_p           0.421530
saf_p_11      0.122913
com_p_11     -0.115073
eng_p_11      0.020254
aca_p_11      0.035155
saf_t_11      0.313810
com_t_11      0.082419
eng_t_11      0.036906
aca_t_11      0.132348
saf_s_11      0.337639
com_s_11      0.187370
eng_s_11      0.213822
aca_s_11      0.339435
saf_tot_11    0.318753
com_tot_11    0.077310
eng_tot_11    0.100102
aca_tot_11    0.190966
Name: sat_score, dtype: float64

In [24]:
# Let's focus on the correlations between SAT score and the survey fields.
# First, let's remove DBN since it's a unique identifier, which is not useful for correlation.
survey_fields.remove("DBN")

ValueError: list.remove(x): x not in list

In [ ]:
survey_fields.remove("DBN")

In [ ]:
correlations = combined.corr()
correlations = correlations["sat_score"].abs()
print(correlations.sort_values(ascending=False).head(20))


In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')
combined.corr()["sat_score"][survey_fields] #.plot.bar()

In [ ]:
survey_fields_corr = pd.DataFrame(combined.corr()['sat_score'][survey_fields]).reset_index()
survey_fields_corr = survey_fields_corr.rename(columns={'index': 'field', 'sat_score':'sat_score_corr'})
alt.Chart(survey_fields_corr).mark_bar().encode(
    x='field',
    y='sat_score_corr'
)


In [ ]:
survey_fields_corr = pd.DataFrame(combined.corr()['sat_score'][survey_fields]).reset_index()
survey_fields_corr = survey_fields_corr.rename(columns={'index': 'field', 'sat_score':'sat_score_corr'})
alt.Chart(survey_fields_corr).mark_bar().encode(
    x='field',
    y='sat_score_corr'
)
